# Prototypes and Criticisms using MMD-critic

We need to get the kernel matrix of the input data for the selection of prototypes and criticisms.

The kernel matrix has a shape of the number of records times the number of records.

The full binned dataframe can be used to create a kernel matrix that is 90 GB in memory.

So we randomly sample 10 percent of the binned data with near perfect balance of records per class.

Use the functions to select prototypes and criticisms from the MMD-critic github repo.

In [ ]:
!git clone https://github.com/BeenKim/MMD-critic.git

In [ ]:
import numpy as np
import sys

In [1]:
# Load x_bins into memory

In [ ]:
sampling_ratio = 0.1

sample_indices = x_bins.sample(frac=sampling_ratio, random_state=1).index
x_bins_sampled = x_bins.loc[sample_indices]
x_bins.sampled = x_bins_sampled.drop(columns=['Sample_id'])
y_raw_sampled = y_raw.loc[sample_indices]

print(y_raw_sampled.Class_Label.value_counts())

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel

def compute_kernel_matrix(df, gamma=0.1):
    data = df.to_numpy()
    K = rbf_kernel(data, gamma=gamma)
    return K

K = compute_kernel_matrix(x_bins_sampled, gamma=0.0001)

In [ ]:
num_prototypes=10
num_criticisms=10

In [2]:
# greedy_select_protos copied function from MMD-critic/mmd.py

##############################################################################################################################
# Function choose m of all rows by MMD as per kernelfunc
# ARGS:
# K : kernel matrix
# candidate_indices : array of potential choices for selections, returned values are chosen from these  indices
# m: number of selections to be made
# is_K_sparse:  True means K is the pre-computed  csc sparse matrix? False means it is a dense matrix.
# RETURNS: subset of candidate_indices which are selected as prototypes
##############################################################################################################################

def greedy_select_protos(K, candidate_indices, m, is_K_sparse=False):

    if len(candidate_indices) != np.shape(K)[0]:
        K = K[:,candidate_indices][candidate_indices,:]

    n = len(candidate_indices)

    # colsum = np.array(K.sum(0)).ravel() # same as rowsum
    if is_K_sparse:
        colsum = 2*np.array(K.sum(0)).ravel() / n
    else:
        colsum = 2*np.sum(K, axis=0) / n

    selected = np.array([], dtype=int)
    value = np.array([])
    for i in range(m):
        maxx = -sys.float_info.max
        argmax = -1
        candidates = np.setdiff1d(range(n), selected)

        s1array = colsum[candidates]
        if len(selected) > 0:
            temp = K[selected, :][:, candidates]
            if is_K_sparse:
                # s2array = temp.sum(0) *2
                s2array = temp.sum(0) * 2 + K.diagonal()[candidates]

            else:
                s2array = np.sum(temp, axis=0) *2 + np.diagonal(K)[candidates]

            s2array = s2array/(len(selected) + 1)

            s1array = s1array - s2array

        else:
            if is_K_sparse:
                s1array = s1array - (np.abs(K.diagonal()[candidates]))
            else:
                s1array = s1array - (np.abs(np.diagonal(K)[candidates]))

        argmax = candidates[np.argmax(s1array)]
        # print("max %f" %np.max(s1array))

        selected = np.append(selected, argmax)
        # value = np.append(value,maxx)
        KK = K[selected, :][:, selected]
        if is_K_sparse:
            KK = KK.todense()

        inverse_of_prev_selected = np.linalg.inv(KK)  # shortcut

    return candidate_indices[selected]

In [ ]:
prototypes = greedy_select_protos(K=K, candidate_indices=np.arange(len(K)), m=num_prototypes, is_K_sparse=False)

for prototype in prototypes:
    print(f"Looking into prototype {prototype} with class label {y_raw_sampled['Class_Label'].iloc[prototype]}")
    # print(x_bins_sampled.iloc[prototype])

In [3]:
# select_criticism_regularized function copied from MMD-critic/mmd.py

##############################################################################################################################
# function to select criticisms
# ARGS:
# K: Kernel matrix
# selectedprotos: prototypes already selected
# m : number of criticisms to be selected
# reg: regularizer type.
# is_K_sparse:  True means K is the pre-computed  csc sparse matrix? False means it is a dense matrix.
# RETURNS: indices selected as criticisms
##############################################################################################################################
def select_criticism_regularized(K, selectedprotos, m, reg='logdet', is_K_sparse=True):

    n = np.shape(K)[0]
    if reg in ['None','logdet','iterative']:
        pass
    else:
        print("wrong regularizer :" + reg)
        exit(1)
    options = dict()

    selected = np.array([], dtype=int)
    candidates2 = np.setdiff1d(range(n), selectedprotos)
    inverse_of_prev_selected = None  # should be a matrix

    if is_K_sparse:
        colsum = np.array(K.sum(0)).ravel()/n
    else:
        colsum = np.sum(K, axis=0)/n

    for i in range(m):
        maxx = -sys.float_info.max
        argmax = -1
        candidates = np.setdiff1d(candidates2, selected)

        s1array = colsum[candidates]

        temp = K[selectedprotos, :][:, candidates]
        if is_K_sparse:
            s2array = temp.sum(0)
        else:
            s2array = np.sum(temp, axis=0)

        s2array = s2array / (len(selectedprotos))

        s1array = np.abs(s1array - s2array)
        if reg == 'logdet':
            if inverse_of_prev_selected is not None: # first call has been made already
                temp = K[selected, :][:, candidates]
                if is_K_sparse:
                    temp2 = temp.transpose().dot(inverse_of_prev_selected)
                    regularizer = temp.transpose().multiply(temp2)
                    regcolsum = regularizer.sum(1).ravel()# np.sum(regularizer, axis=0)
                    regularizer = np.abs(K.diagonal()[candidates] - regcolsum)

                else:
                    # hadamard product
                    temp2 = np.array(np.dot(inverse_of_prev_selected, temp))
                    regularizer = temp2 * temp
                    regcolsum = np.sum(regularizer, axis=0)
                    regularizer = np.log(np.abs(np.diagonal(K)[candidates] - regcolsum))
                s1array = s1array + regularizer
            else:
                if is_K_sparse:
                    s1array = s1array - np.log(np.abs(K.diagonal()[candidates]))
                else:
                    s1array = s1array - np.log(np.abs(np.diagonal(K)[candidates]))
        argmax = candidates[np.argmax(s1array)]
        maxx = np.max(s1array)

        selected = np.append(selected, argmax)
        if reg == 'logdet':
            KK = K[selected,:][:,selected]
            if is_K_sparse:
                KK = KK.todense()

            inverse_of_prev_selected = np.linalg.inv(KK) # shortcut
        if reg == 'iterative':
            selectedprotos = np.append(selectedprotos, argmax)

    return selected

In [ ]:
criticisms = select_criticism_regularized(K, selectedprotos=prototypes, m=num_criticisms, is_K_sparse=False)

for criticism in criticisms:
    print(f"Looking into criticism {criticism} with class label {y_raw_sampled['Class_Label'].iloc[criticism]}")
    # print(x_bins_sampled.iloc[criticism])

In [ ]:
prototype_indices = [x_bins_sampled.index[prototype] for prototype in prototypes]
print(prototype_indices)

criticism_indices = [x_bins_sampled.index[criticism] for criticism in criticisms]
print(criticism_indices)